In [168]:
with open('day16_inputs.txt') as f:
    content = [l.strip('\n') for l in f]

content

['departure location: 31-201 or 227-951',
 'departure station: 49-885 or 892-961',
 'departure platform: 36-248 or 258-974',
 'departure track: 37-507 or 527-965',
 'departure date: 37-331 or 351-970',
 'departure time: 38-370 or 382-970',
 'arrival location: 33-686 or 711-960',
 'arrival station: 46-753 or 775-953',
 'arrival platform: 34-138 or 154-959',
 'arrival track: 26-167 or 181-961',
 'class: 43-664 or 675-968',
 'duration: 47-603 or 620-954',
 'price: 40-290 or 313-972',
 'route: 37-792 or 799-972',
 'row: 32-97 or 115-954',
 'seat: 25-916 or 942-966',
 'train: 39-572 or 587-966',
 'type: 25-834 or 858-953',
 'wagon: 48-534 or 544-959',
 'zone: 47-442 or 463-969',
 '',
 'your ticket:',
 '127,83,79,197,157,67,71,131,97,193,181,191,163,61,53,89,59,137,73,167',
 '',
 'nearby tickets:',
 '196,877,117,24,56,120,416,948,677,238,628,285,326,813,155,659,91,717,287,279',
 '354,870,624,158,473,361,565,896,873,549,740,714,380,827,880,871,271,187,66,662',
 '563,659,90,894,686,496,245,500

#### Task is to validate train tickets

Input consists of 3 things:
- A set of rules, these specify, for each field on a ticket, what the valid range of values are. The order of the rules doesn't necessarily reflect the order of the fields on the tickets
- Field values for your ticket
- Field values for other people's tickets (nearby)

In [169]:
import copy

# Parse rules
rules = copy.deepcopy(content[:20])#[:20])
parsed_rules = []
for r in rules:
    field, value_ranges = r.split(': ')
    value_ranges = [s.split('-') for s in value_ranges.split(' or ')]
    int_value_ranges = [[int(start),int(end)] for (start,end) in value_ranges]
    parsed_rules.append([field, int_value_ranges])
    
dict_rules = dict(parsed_rules)

# Parse ticket 
my_ticket = [int(n) for n in copy.deepcopy(content[22]).split(',')] #22

# Parse nearby tickets
nearby_tickets = [list(map(lambda n: int(n),l.split(','))) for l in copy.deepcopy(content[25:])] #25

### Part 1

#### First task is to spot invalid tickets in the list of nearby tickets.

For example for these rules:
- class: 1-3 or 5-7
- row: 6-11 or 33-44
- seat: 13-40 or 45-50

The number '4' is invalid for any of these fields, so any ticket with a 4 in any of the fields must be invalid. Again, we don't know what the ordering of the fields are.


In [170]:
# Get valid number ranges
valid_numbers_with_duplicates = []

for rule in dict_rules:
    num_ranges = dict_rules[rule]
    for num_range in num_ranges:
        # Ensure we include the end number in each range
        valid_numbers_with_duplicates.extend(list(range(num_range[0],num_range[1]+1)))
    
valid_numbers = list(set(valid_numbers_with_duplicates))

# Scan tickets and capture errors
scanning_error_rate = 0
valid_tickets = []
for i in range(len(nearby_tickets)):
    ticket = nearby_tickets[i]
    valid_ticket = True
    for num in ticket:
        if num not in valid_numbers:
            scanning_error_rate += num
            valid_ticket = False
    
    if valid_ticket:
        # Use for part 2
        valid_tickets.append(nearby_tickets[i])
        
scanning_error_rate

20091

### Part 2

#### Second task is to eliminate invalid tickets, then determine which field is which

Once you've done this, return the multiple of the **departure** fields on **your** ticket

In [171]:
dict_rules

{'departure location': [[31, 201], [227, 951]],
 'departure station': [[49, 885], [892, 961]],
 'departure platform': [[36, 248], [258, 974]],
 'departure track': [[37, 507], [527, 965]],
 'departure date': [[37, 331], [351, 970]],
 'departure time': [[38, 370], [382, 970]],
 'arrival location': [[33, 686], [711, 960]],
 'arrival station': [[46, 753], [775, 953]],
 'arrival platform': [[34, 138], [154, 959]],
 'arrival track': [[26, 167], [181, 961]],
 'class': [[43, 664], [675, 968]],
 'duration': [[47, 603], [620, 954]],
 'price': [[40, 290], [313, 972]],
 'route': [[37, 792], [799, 972]],
 'row': [[32, 97], [115, 954]],
 'seat': [[25, 916], [942, 966]],
 'train': [[39, 572], [587, 966]],
 'type': [[25, 834], [858, 953]],
 'wagon': [[48, 534], [544, 959]],
 'zone': [[47, 442], [463, 969]]}

In [172]:
dict_field_possible_locations = {}
fields = list(dict_rules.keys())

# Go through 1 position at a time in all tickets, then validate against rules
# Once we have a match (field valid for that position in all valid tickets), 
# add that field as a valid candidate for that field

for i in range(len(fields)):
    values_at_position = []
    
    # We already got rid of invalid tickets in step 1
    for ticket in valid_tickets:
        values_at_position.append(ticket[i])
    
    # Figure out which fields this position could take
    valid_fields = []
    for field in fields:
        value_ranges = dict_rules[field]
        
        # Turn into list of possible numbers
        field_valid_values_w_dupes = []
        for start_value, end_value in value_ranges:
            field_valid_values_w_dupes.extend(list(range(start_value, end_value+1)))
        field_valid_values = list(set(field_valid_values_w_dupes))
        
        # Check all values at this position are valid for the field
        if all([v in field_valid_values for v in values_at_position]):
            valid_fields.append(field)
            
    dict_field_possible_locations[i] = valid_fields

dict_field_possible_locations

{0: ['departure location',
  'departure station',
  'departure platform',
  'departure track',
  'departure date',
  'departure time',
  'arrival location',
  'arrival station',
  'arrival platform',
  'arrival track',
  'class',
  'duration',
  'price',
  'route',
  'row',
  'seat',
  'train',
  'type',
  'wagon',
  'zone'],
 1: ['departure location',
  'departure platform',
  'departure track',
  'departure date',
  'arrival location',
  'arrival station',
  'arrival platform',
  'arrival track',
  'class',
  'row',
  'seat',
  'train',
  'zone'],
 2: ['arrival location',
  'arrival station',
  'arrival platform',
  'class',
  'row',
  'seat',
  'train',
  'zone'],
 3: ['departure location',
  'departure station',
  'departure platform',
  'departure track',
  'departure date',
  'departure time',
  'arrival location',
  'arrival station',
  'arrival platform',
  'arrival track',
  'class',
  'price',
  'route',
  'row',
  'seat',
  'train',
  'type',
  'zone'],
 4: ['departure locat

In [173]:
# Now loop through this dictionary and keep entries where only 1 field is possible
# Then remove that field from all other position potential fields

dict_field_locations = dict_field_possible_locations.copy()


for i in range(100):
    for key in dict_field_locations:
        if len(dict_field_locations[key]) == 1:

            # Turn into scalar
            field = dict_field_locations[key][0]

            # Remove from all other lists
            for sub_key in dict_field_locations:
                if (key == sub_key) or (len(dict_field_locations[sub_key]) == 1):
                    continue
                else:
                    current_values = dict_field_locations[sub_key]
                    dict_field_locations[sub_key] = [value for value in current_values if value != field]
                    keys_updated == True

dict_field_locations

{0: ['duration'],
 1: ['departure date'],
 2: ['arrival station'],
 3: ['price'],
 4: ['departure time'],
 5: ['train'],
 6: ['class'],
 7: ['departure station'],
 8: ['arrival track'],
 9: ['arrival location'],
 10: ['row'],
 11: ['seat'],
 12: ['departure location'],
 13: ['departure platform'],
 14: ['type'],
 15: ['arrival platform'],
 16: ['zone'],
 17: ['departure track'],
 18: ['wagon'],
 19: ['route']}

In [177]:
departure_keys = [k for k in dict_field_locations.keys() if dict_field_locations[k][0].startswith('departure')]

num = 1
for k in departure_keys:
    num*=my_ticket[k]
num

2325343130651